# 1) Données

In [1]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%36)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
#dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = 36
#colsDataOperateurs = len(dataOperateurs.axes[1])
#nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
#competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
rho = [1,1,1,1,0,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1,1,0,1,1,0,1]
cardP = 36

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 36
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,0,0,1,0,1,1,0,0,1,1,0,1,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,0,0,1,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,0,0,1,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,1,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,1,1,0,1,1,1,1,1,0,1,1,0,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,1,0,0,0,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1]]


# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
#ATTENTION JE CONSERVE LES MEMES PERMUATIONS QUE DANS LE DIAPO DE LA REUNION DU 27/04
#sigma = [
#[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
#[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1]
#]

fac = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


d = [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        profils[i] = cardProfils
        cardProfils = cardProfils + 1
        
        
print(profils)
#print(cardProfils)
N = np.zeros(cardProfils)
for i in range(cardO):
    N[int(profils[i])] += 1
#print(N)

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein

delta = np.full((cardProfils, cardJ), 1)
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    if d[index] != -1:
        d_i = d[index]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[profil][j_s] = 0
            
kappaProfils = []
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    kappaProfils.append(kappa[index])
    
creneaux = [5,1,4,1,0,4,0,4,3,4,5,4,1,0,4,1,4,0,1,0,5,4,1,0,3,4,1,0,1,4,5,0,1,4,0,2]
    
cardCreneaux = len(np.unique(creneaux))
alpha = []
for p in range(cardP):
    ligne = []
    for cren in range(cardCreneaux):
        if creneaux[p] == cren:
            ligne.append(1)
        else:
            ligne.append(0)
    alpha.append(ligne)
    
print(alpha)

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
#print(delta)
#print(o)
#print(H[1])
#print(kappaProfils)
#print(delta[0])
#print(creneaux[21])
pen = []
for poste1 in range(cardP):
    ligne = []
    val1 = 0
    val2 = 0
    for poste2 in range(cardP):
        if poste1 == poste2 or (creneaux[poste1] == 0 and creneaux[poste2] == 0) or (creneaux[poste1] == 0 and creneaux[poste2] != 0):
            ligne.append(0)
        elif creneaux[poste1] != 0 and creneaux[poste2] == 0: #rouleur
            ligne.append(10) #coût pour passer d'un poste non rouleur à rouleur
        else:
            if creneaux[poste1] == 1:
                val1 = 7
            if creneaux[poste1] == 2:
                val1 = 8
            if creneaux[poste1] == 3:
                val1 = 9
            if creneaux[poste1] == 4:
                val1 = 10.5
            if creneaux[poste1] == 5:
                val1 = 12.45
            if creneaux[poste2] == 1:
                val2 = 7
            if creneaux[poste2] == 2:
                val2 = 8
            if creneaux[poste2] == 3:
                val2 = 9
            if creneaux[poste2] == 4:
                val2 = 10.5
            if creneaux[poste2] == 5:
                val2 = 12.45
            ligne.append(abs(val1-val2))
    pen.append(ligne)
#print(pen)

[0. 0. 0. 0. 0. 1. 1. 2. 0. 0. 3. 0. 0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 4. 5.
 5. 5. 6. 5. 7. 5. 7. 7. 5. 8. 7. 7.]
[[0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]]


# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for prof in range(0,cardProfils):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[prof,j,p]=LpVariable(f"x({prof,j,p})", cat=LpBinary) #LpBinary

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for prof in range(0,cardProfils):
    for r in range(0,cardR):    
        y[prof,r]=LpVariable(f"y({prof,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for prof in range(0,cardProfils):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[prof,s,p]=LpVariable(f"z({prof,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for p in range(0,cardP):
    for s in range(0,cardS):    
        u[p,s]=LpVariable(f"u({p,s})", cat=LpBinary)

print("Fin de création des variables")

Fin de création des variables


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModeleProfils36", LpMinimize)

#prob += sum(u[p,s] for p in range(0,cardP) for s in range(0,cardS)), "obj"

print("Fin de création du modèle")

Fin de création du modèle


## 2.3) Contraintes

In [4]:
#contrainte 1 Un profil opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[i,j,p] <= delta[i][j]*kappaProfils[i][p]*o[j], f"C1_{i,j,p}")

#contrainte 2 Un profil opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappaProfils[i][p], f"C2_{i,s,p}")

#contrainte 3 Chaque jour, un profil opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for i in range(0,cardProfils):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                a = 0
                #prob += x[i,j,p] <= z[i,s,p] + sum((1 - rho[p2])*z[i,s,p2] for p2 in range(0,cardP)), f"C3_{i,j,p}"

for j in range(cardJ):
    s = j // 7
    for i in range(cardProfils):
        for p in range(cardP):
            if rho[p] == 1 and o[j] == 1 and delta[i][j] == 1:
                #print("p=",p)
                a=0
                prob += z[i,s,p] <= x[i,j,p], f"C3bis_{j,i,p}"

#contrainte 4 Chaque jour, un profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob += sum(x[i,j,p] for p in range(0,cardP)) <= N[i], f"C4_{i,j}"

#contrainte 5 Chaque jour, chaque poste non rouleur est assuré par exactement un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            if fac[p] == 0:
                prob += sum(x[i,j,p] for i in range(cardProfils)) == 1, f"C5_{j,p}"
            else:
                prob += sum(x[i,j,p] for i in range(cardProfils)) <= 1, f"C5_{j,p}"
            
#contrainte 6 Chaque jour, les postes rouleurs sont occupés par au plus un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[i,j,p] for i in range(0,cardProfils)) <= 1, f"C6_{j,p}"

#contrainte 7 Chaque semaine, chaque profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) <= N[i], f"C7_{i,s}"

#contrainte 8 Chaque semaine, chaque poste est assuré par un profil opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        if fac[p] == 0:
            prob += sum(z[i,s,p] for i in range(0,cardProfils)) == 1, f"C8_{s,p}"
        else:
            prob += sum(z[i,s,p] for i in range(0,cardProfils)) <= 1, f"C8_{s,p}"

#contrainte 9 Chaque profil opérateur est affecté à autant de positions dans le roulement que son nombre de membres

for i in range(0,cardProfils):
    prob += sum(y[i,r] for r in range(0,cardR)) == N[i], f"C9_{i}"

#contrainte 10 Chaque position dans le roulement est associée à un profil opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardProfils)) == 1, f"C10_{r}"

#contrainte 11 Un profil opérateur peut permuter de poste si leurs horaires sont compatibles

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            a = 0
            #prob += z[i,s,p] <= sum(sigma[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C11_{i,s,p}"

#contrainte 12 Lorsqu'un poste est occupé par un autre profil opérateur que celui prévu initialement,
#son score d'insatisfaction augmente
for s in range(0,cardS):
    for p in range(0,cardP):
        for i in range(cardProfils):
            #prob += u[p,s] >= (kappaProfils[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p]), f"C12_{i,s,p}"
            for p2 in range(cardP):
                a = 0
                prob += u[p,s] >= (kappaProfils[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p])*pen[p][p2], f"C12_{i,s,p,p2}"
            
#contrainte David anti permutation interdite
for s in range(cardS):
    for i in range(cardProfils):
        for c in range(cardCreneaux):
            a = 0
            #prob += sum(alpha[p][c]*z[i,s,p] for p in range(cardP)) == sum(alpha[p][c]*y[i,rFunction(s,p)] for p in range(cardP)), f"C13_{s,i,c}"

for j in range(cardJ):
    s = j // 7
    for i in range(cardProfils):
        a = 0
        #prob += sum(x[i,j,p]*rho[p] for p in range(cardP)) <= sum(z[i,s,p]*rho[p] for p in range(cardP)), f"C14_{j,i,c}"

print("Fin de création des contraintes")

Fin de création des contraintes


## 2.4) Résolution du problème

In [5]:
import time
import cplex
from cplex.callbacks import MIPInfoCallback
log = []

class MyCallback(MIPInfoCallback):

    def __call__(self):
        current = time.time()
        log.append([self.get_incumbent_objective_value(), current - start])

pathFile = "Modeles/ModeleExactProfils36.lp"
prob.writeLP(pathFile)

c = cplex.Cplex()
c.parameters.timelimit.set(18000.0)
c.parameters.mip.cuts.cliques.set(3)
c.register_callback(MyCallback)
c.read(pathFile)

start = time.time()
c.solve()
#solution = c.solution
#print(solution.get_objective_value())
end = time.time()

duree = end - start
print()
print("Temps de résolution : " + str(duree))
print("Valeur objectif = ", c.solution.get_objective_value())

print()
print("log")
for i in range(len(log)):
    print(log[i])

newLog = []
currentSol = 10000
for i in range(len(log)):
    if log[i][0] < currentSol:
        currentSol = log[i][0]
        newLog.append(log[i])
newLog.append(log[len(log)-1])
 
print()
print("newLog")
for i in range(len(newLog)):
    print(newLog[i])
    
for i in range(len(newLog)):
    str1 = str(newLog[i][0]).replace('.', ',')
    print(str1)
print()
for i in range(len(newLog)):
    str1 = str(newLog[i][1]).replace('.', ',')
    print(str1)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Cuts_Cliques                        3
CPXPARAM_TimeLimit                               18000
Legacy callback                                  i
Tried aggregator 3 times.
MIP Presolve eliminated 531648 rows and 66277 columns.
MIP Presolve modified 461736 coefficients.
Aggregator did 8460 substitutions.
Reduced MIP has 24993 rows, 20196 columns, and 98244 nonzeros.
Reduced MIP has 20196 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.72 sec. (596.72 ticks)
Probing time = 0.07 sec. (8.36 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 24993 rows, 20196 columns, and 98244 nonzeros.
Reduced MIP has 20196 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.17 sec. (139.69 ticks)
Probing time = 0.06 sec. (7.80 ticks)
Clique table members: 57798.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic 

CPLEX Error  1217: No solution exists.



Temps de résolution : 17.5656635761261


CplexSolverError: CPLEX Error  1217: No solution exists.

# 3) Affichage de la solution

In [ ]:
print("objectif =", c.solution.get_objective_value())
for s in range(cardS):
    for p in range(cardP):
        val = c.solution.get_values("u(("+str(p)+",_"+str(s)+"))")
        if val != 0:
            print("u[",p,",",s,"] =", val)

print()
for i in range(cardProfils):
    for r in range(cardR):
        val = c.solution.get_values("y(("+str(i)+",_"+str(r)+"))")
        if val != 0:
            print("y[",i,",",r,"] =", val)
        
print()
for s in range(cardS):
    print()
    for i in range(cardProfils):
        for p in range(cardP):
            val = c.solution.get_values("z(("+str(i)+",_"+str(s)+",_"+str(p)+"))")
            if val != 0:
                print("z[",i,",",s,",",p,"] =", val)

#print(c.variables.get_names())
#solver.solution.get_objective_value()
#print(objval)

# 4) Conversion de la solution vers des profils individuels (pour texte et CSV)

In [ ]:
groupesProfil = []
for profil in range(cardProfils):
    groupeProfil = []
    for i in range(cardO):
        if profils[i] == profil:
            groupeProfil.append(i)
    groupesProfil.append(groupeProfil)

print("groupesProfil = ", groupesProfil)

vecteurY = np.zeros(cardO)
newY = np.zeros((cardO,cardR))
for prof in range(cardProfils):
    groupeProfil = groupesProfil[prof]
    roulements = []
    for r in range(cardR):
        if c.solution.get_values("y(("+str(prof)+",_"+str(r)+"))") == 1:
            roulements.append(r)
    #print(groupeProfil)
    #print(roulements)
    for index in range(len(groupeProfil)):
        newY[groupeProfil[index]][roulements[index]] = 1
        vecteurY[groupeProfil[index]] = roulements[index]

#print(vecteurY)
#print("newY =", newY)
print("vecteurY =", vecteurY)

print()
newZ = []
ligne = []
for i in range(cardO):
    ligne.append(-1)
for s in range(cardS):
    newZ.append(ligne.copy())

#print(newZ)

for profil in range(cardProfils):
    #On regarde qui appartient au groupe
    groupeProfil = groupesProfil[profil].copy()
    for s in range(cardS):
        #print("semaine", s, "profil", profil)
        affectProfil = []
        #Pour chaque semaine, on regarde quels postes doivent prendre en charge le groupe
        for p in range(cardP):
            if int(c.solution.get_values("z(("+str(profil)+",_"+str(s)+",_"+str(p)+"))")) == 1:
                affectProfil.append(p)
        #print("Postes à affecter :", affectProfil)
        #print("groupeProfil", groupeProfil)
        #Pour chaque personne appartenant au groupe, on lui donne le poste qu'elle doit faire si possible
        operateursLibres = groupeProfil.copy() #copie pour savoir qui n'a pas encore de poste
        for index in range(len(groupeProfil)):
            ope = groupeProfil[index]
            #print("ope =", ope)
            posteTheoriqueOpe = (vecteurY[ope] + s) % cardP
            #print("posteTheoriqueOpe =", posteTheoriqueOpe)
            if posteTheoriqueOpe in affectProfil:
                affectProfil.remove(posteTheoriqueOpe)
                operateursLibres.remove(ope)
                newZ[s][ope] = posteTheoriqueOpe
        if len(affectProfil) >= 1:
            #print("reste", operateursLibres, "pour", affectProfil)
            while len(affectProfil) > 0:
                poste = affectProfil[0]
                #print("on veut affecter le poste", poste)
                #print("affectProfil", affectProfil)
                #print("operateursLibres", operateursLibres)
                creneauPoste = creneaux[poste]
                trouve = False
                index2 = 0
                while index2 < len(operateursLibres) and not trouve:
                    ope = operateursLibres[index2]
                    posteTheorique = int((vecteurY[ope] + s) % cardP)
                    #print("posteTheorique =", posteTheorique)
                    #print("on compare", creneaux[posteTheorique], "et", creneauPoste)
                    if creneaux[posteTheorique] == creneauPoste:
                        trouve = True
                        newZ[s][ope] = poste
                        affectProfil.remove(poste)
                        operateursLibres.remove(ope)
                    else:
                        index2 += 1
                
                    
print("newZ = ")
for index in range(len(newZ)):
    print(newZ[index])
#print(newZ[0])

newX = []
ligne = []
for i in range(cardO):
    ligne.append(-1)
for j in range(cardJ):
    newX.append(ligne.copy())
    
for j in range(cardJ):
    #print("j =",j)
    semaineJ = j // 7
    for profil in range(cardProfils):
        #on dresse la liste des postes que ce profil doit prendre en charge ce jour
        postesVacants = []
        for p in range(cardP):
            if int(c.solution.get_values("x(("+str(profil)+",_"+str(j)+",_"+str(p)+"))")) == 1:
                postesVacants.append(p)
        #print("jour", j, "profil", profil, "postesVacants", postesVacants)
        #on dresse la liste des opérateurs libres ce jour
        if delta[profil][j] == 1 and o[j] == 1:
            groupeProfil = groupesProfil[profil]
        else:
            groupeProfil = []
        operateursLibres = groupeProfil.copy()
        
        #print("avant groupeProfil =", groupeProfil)
        #print("avant operateursLibres =", operateursLibres)
        
        #pour chaque opérateur du groupe, si son affectation hebdo est la même que celle du jour pour son profil on l'y affecte
        for index in range(len(groupeProfil)):
            ope = groupeProfil[index]       
            affectHebdo = newZ[semaineJ][ope]
            profilOpe = profils[ope]
            if int(c.solution.get_values("x(("+str(int(profilOpe))+",_"+str(j)+",_"+str(int(affectHebdo))+"))")) == 1:
                newX[j][ope] = affectHebdo
                operateursLibres.remove(ope)
                postesVacants.remove(affectHebdo)
        if len(operateursLibres) > 0:
            #print("operateursLibres", operateursLibres)
            #print("postesVacants", postesVacants)
            for index in range(len(operateursLibres)):
                newX[j][operateursLibres[index]] = postesVacants[index]
        #if len(operateursLibres) > 0 and j%7 != 5 and j%7 != 6:
            #print("jour", j, "profil", profil, "IL RESTE", operateursLibres, "pour les postes", postesVacants)
    
print("newX = ")
#print(newX)           

    

# 4) Extraction de la solution au format texte

In [ ]:
for i in range(cardProfils):
    for j in range(cardJ):
        for p in range(cardP):
            if c.solution.get_values("x(("+str(i)+",_"+str(j)+",_"+str(p)+"))") != 0:
                print("x(("+str(i)+",_"+str(j)+",_"+str(p)+"))")

In [ ]:
with open("solutionsTexte/solutionModeleExactProfils", "w") as file_object:
        file_object.write(str(cardO) + "\n")           #cardO
        file_object.write(str(cardP) + "\n")           #cardP
        file_object.write(str(cardR) + "\n")           #cardR
        file_object.write(str(cardS) + "\n")           #cardS
        ligne = ""
        for p in range(0,cardP):
            ligne = ligne + str(rho[p])
        file_object.write(ligne + "\n")                #rho
        
        ligne = ""
        for j in range(0,cardJ):
            ligne = ligne + str(int(o[j]))
        file_object.write(ligne + "\n")                #o_j
        
        for i in range(0,cardO):
            ligne = ""
            for p in range(0,cardP):
                ligne = ligne + str(kappa[i][p])
            file_object.write(ligne + "\n")            #kappa_ip
            
        for p in range(0,cardP):
            ligne = ""
            for p2 in range(0,cardP):
                ligne = ligne + str(sigma[p][p2])
            file_object.write(ligne + "\n")            #sigma_pp'
            
        for i in range(0,cardO):
            ligne = ""
            for j in range(0,cardJ):
                ligne = ligne + str(delta[int(profils[i])][j])
            file_object.write(ligne + "\n")            #delta_ij
        
        ##Avant
        ##file_object.write("x" + "\n")
        #for i in range(0,cardO):
        #    #file_object.write(str(i) + "\n")
        #    for j in range(0,cardJ):
        #        ligne = ""
        #        for p in range(0,cardP):
        #            ligne = ligne + str(int(c.solution.get_values("x(("+str(int(profils[i]))+",_"+str(j)+",_"+str(p)+"))")))
        #        file_object.write(ligne + "\n")        #x_ijp
        
        #file_object.write("y" + "\n")
        
        for i in range(0,cardO):
            for j in range(0,cardJ):
                ligne = ""
                poste = newX[j][i]
                if poste == -1:
                    for p in range(0,cardP):
                        ligne += "0"
                else:
                    for p in range(0,cardP):
                        if p == poste:
                            ligne += "1"
                        else:
                            ligne += "0"
                file_object.write(ligne + "\n")        #x_ijp
        
        for i in range(0,cardO):
            ligne = ""
            for r in range(0,cardR):
                if newY[i][r] == 1:
                    ligne += "1"
                else:
                    ligne += "0"
            file_object.write(ligne + "\n")            #y_ir
        
        #file_object.write("z" + "\n")
        for i in range(0,cardO):
            for s in range(0,cardS):
                ligne = ""
                for p in range(0,cardP):
                    if newZ[s][i] == p:
                        ligne += "1"
                    else:
                        ligne += "0"
                    #ligne = ligne + str(int(c.solution.get_values("z(("+str(int(profils[i]))+",_"+str(s)+",_"+str(p)+"))")))
                file_object.write(ligne + "\n")        #z_isp
        
        file_object.write((str(int(c.solution.get_objective_value()))) + "\n")
        
print("Solution texte extraite")

# 5) Extraction de la solution au format CSV

In [ ]:
import csv
with open('CSV/planningExactProfils.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if newX[j][i] == p:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)
print("Planning CSV exporté")

In [ ]:
print("Fin de programme")